<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/explore_nbs/Explore_10_nlp_TPU_Comparison_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide_input
#colab
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#hide
#colab
!curl -s https://course.fast.ai/setup/colab | bash 

Updating fastai...
Done.


Dont run for GPU

In [ ]:
# hide_output
# colab
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  89736      0 --:--:-- --:--:-- --:--:-- 89736
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 2.8MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     


Install fastai2 and the fastai_xla_extensions packages

In [ ]:
#hide_output
#colab
!pip install fastai2 --upgrade > /dev/null

In [ ]:
#hide_output
#ci
!pip install git+https://github.com/butchland/fastai_xla_extensions > /dev/null

  Running command git clone -q https://github.com/butchland/fastai_xla_extensions /tmp/pip-req-build-vm2_bwux


In [ ]:
%cd /content/drive/My\ Drive/course-v4

/content/drive/My Drive/course-v4


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 1.1MB 5.0MB/s 
     |████████████████████████████████| 40kB 4.6MB/s 
     |████████████████████████████████| 92kB 6.4MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 2.7MB 10.9MB/s 


In [ ]:
%cd nbs

/content/drive/My Drive/course-v4/nbs


### Import the libraries
Import the fastai2 and fastai_xla_extensions libraries

In [ ]:
#colab
import fastai_xla_extensions.core

In [ ]:
from fastai2.text.all import *

In [ ]:
#hide
from utils import *
from IPython.display import display,HTML

In [ ]:
%cd /content

/content


# NLP Deep Dive: RNNs

## Text Preprocessing

### Tokenization

### Word Tokenization with fastai

In [ ]:

path = untar_data(URLs.IMDB)

In [ ]:
Path.BASE_PATH = path; path.ls()

(#7) [Path('README'),Path('unsup'),Path('train'),Path('tmp_lm'),Path('tmp_clas'),Path('imdb.vocab'),Path('test')]

In [ ]:
default_device()

device(type='xla', index=1)

## Training a Text Classifier

### Language Model Using DataBlock

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])


In [ ]:

datablock = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
)

In [ ]:
dls_lm = datablock.dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.device

device(type='xla', index=1)

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj up the xxmaj creek is right up there with my most liked movies . xxmaj i 'm crying for a xxup dvd . xxmaj what kind of sense of humor does n't think "" take no prisoners "" was n't funny when the gals are "" going after xxmaj it . "" xxmaj the bazooka shooting at the noisy crow was funny . xxmaj there is funny rolling over funny in this movie but maybe some of it","xxmaj up the xxmaj creek is right up there with my most liked movies . xxmaj i 'm crying for a xxup dvd . xxmaj what kind of sense of humor does n't think "" take no prisoners "" was n't funny when the gals are "" going after xxmaj it . "" xxmaj the bazooka shooting at the noisy crow was funny . xxmaj there is funny rolling over funny in this movie but maybe some of it is"
1,army ( what xxunk asked her to do ) xxmaj col . xxmaj doran convinced prosecutor to charge her with a premeditated murder ( he knew she ca nt be found quilty ) instead of manslaughter ( there was some possibility to be found quilty ) with taken photos . xxunk also suppress argue - shutting incident to escalate to prevent prosecutor to have any doubt about premeditated murder charge but let it be revealed during the trial what greatly,( what xxunk asked her to do ) xxmaj col . xxmaj doran convinced prosecutor to charge her with a premeditated murder ( he knew she ca nt be found quilty ) instead of manslaughter ( there was some possibility to be found quilty ) with taken photos . xxunk also suppress argue - shutting incident to escalate to prevent prosecutor to have any doubt about premeditated murder charge but let it be revealed during the trial what greatly influenced


### Fine-Tuning the Language Model

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
from fastai2.text.learner import *
from fastai2.text.learner import _get_text_vocab


In [ ]:
def load_ignore_keys(model, wgts):
    "Load `wgts` in `model` ignoring the names of the keys, just taking parameters in order"
    # set_trace()
    sd = model.state_dict()
    for k1,k2 in zip(sd.keys(), wgts.keys()): 
        wdata = wgts[k2].data.clone()
        wdata = wdata.to(default_device())
        sd[k1].data = wdata
    return model.load_state_dict(sd)


In [ ]:

@patch_to(LMLearner)
def load_pretrained(self, wgts_fname, vocab_fname, model=None):
    "Load a pretrained model and adapt it to the data vocabulary."
    # set_trace()
    old_vocab = Path(vocab_fname).load()
    new_vocab = _get_text_vocab(self.dls)
    distrib_barrier()
    wgts = torch.load(wgts_fname, map_location = lambda storage,loc: storage)
    if 'model' in wgts: wgts = wgts['model'] #Just in case the pretrained model was saved with an optimizer
    wgts = match_embeds(wgts, old_vocab, new_vocab)
    load_ignore_keys(self.model if model is None else model, clean_raw_keys(wgts))
    self.freeze()
    return self

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()])

In [ ]:
L(learn.model.parameters())[0].device

device(type='xla', index=1)

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


### Saving and Loading Models

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.893486,3.772820,0.317104,43.502548,12:37
1,3.820479,3.717197,0.323790,41.148880,12:30
2,3.735622,3.659760,0.330321,38.851997,12:09
3,3.677086,3.624794,0.333960,37.516987,12:12
4,3.636646,3.601300,0.337017,36.645859,12:05
5,3.553636,3.584241,0.339355,36.026001,12:04
6,3.507634,3.571892,0.341353,35.583862,12:08
7,3.444101,3.565988,0.342194,35.374371,12:08
8,3.398597,3.566283,0.342647,35.384815,12:11
9,3.375563,3.568166,0.342528,35.451500,12:05


In [ ]:
learn.save_encoder('finetuned')

### Text Generation

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

i liked this movie because of its story and characters . The story line was very strong , very good for a sci - fi film . The main character , Alucard , was very well developed and brought the whole story
i liked this movie because i like the idea of the premise of the movie , the ( very ) convenient virus ( which , when you have to kill a few people , the " evil " machine has to be used to protect


### Creating the Classifier DataLoaders

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos i rate this movie with 3 skulls , only coz the girls knew how to scream , this could 've been a better movie , if actors were better , the twins were xxup ok , i believed they were evil , but the eldest and youngest brother , they sucked really bad , it seemed like they were reading the scripts instead of acting them … . spoiler : if they 're vampire 's why do they freeze the blood ? vampires ca n't drink frozen blood , the sister in the movie says let 's drink her while she is alive … .but then when they 're moving to another house , they take on a cooler they 're frozen blood . end of spoiler \n\n it was a huge waste of time , and that made me mad coz i read all the reviews of how",neg
1,"xxbos i have read all of the xxmaj love xxmaj come xxmaj softly books . xxmaj knowing full well that movies can not use all aspects of the book , but generally they at least have the main point of the book . i was highly disappointed in this movie . xxmaj the only thing that they have in this movie that is in the book is that xxmaj missy 's father comes to xxunk in the book both parents come ) . xxmaj that is all . xxmaj the story line was so twisted and far fetch and yes , sad , from the book , that i just could n't enjoy it . xxmaj even if i did n't read the book it was too sad . i do know that xxmaj pioneer life was rough , but the whole movie was a downer . xxmaj the rating",neg
2,"xxbos xxmaj this , for lack of a better term , movie is lousy . xxmaj where do i start … … \n\n xxmaj cinemaphotography - xxmaj this was , perhaps , the worst xxmaj i 've seen this year . xxmaj it looked like the camera was being tossed from camera man to camera man . xxmaj maybe they only had one camera . xxmaj it gives you the sensation of being a volleyball . \n\n xxmaj there are a bunch of scenes , haphazardly , thrown in with no continuity at all . xxmaj when they did the ' split screen ' , it was absurd . xxmaj everything was squished flat , it looked ridiculous . \n\n xxmaj the color tones were way off . xxmaj these people need to learn how to balance a camera . xxmaj this ' movie ' is poorly made , and",neg


In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

(#10) [228,238,121,290,196,194,533,124,581,155]

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.347427,0.184480,0.929320,00:33


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.247763,0.171683,0.934640,00:37


In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.193377,0.156696,0.941200,00:45


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.172888,0.153770,0.943120,01:01
1,0.161492,0.155567,0.942640,00:57


## Disinformation and Language Models

## Conclusion

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?

### Further Research

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?